<a href="https://colab.research.google.com/github/stevegbrooks/big-portfolio-learner/blob/main/step1b_clean_and_combine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CIS 545 Final Project

## Big Portfolio Learner: Clean Up Technical Indicators Data

### Team members: Steven Brooks & Chenlia Xu

In [ ]:
import random
import numpy as np 
import json
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import datetime
import glob
import seaborn as sns
import re
import os

In [ ]:
%%capture
## If boto3 not already installed uncomment the following:
!pip3 install boto3

In [ ]:
import boto3
from botocore import UNSIGNED

from botocore.config import Config

s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))
s3.Bucket('cis545project').download_file('data/stock_data.zip', 'stock_data.zip')
s3.Bucket('cis545project').download_file('data/technical_data.zip', 'technical_data.zip')

In [ ]:
%%capture

stock_dir = "stock_data"
if not os.path.exists(stock_dir):
  os.makedirs(stock_dir)
!unzip /content/stock_data.zip -d /content/$stock_dir
!rm -f $stock_dir/.gitempty

tech_dir = "technical_data"
if not os.path.exists(tech_dir):
  os.makedirs(tech_dir)
!unzip /content/technical_data.zip -d /content/$tech_dir
!rm -f $tech_dir/.gitempty

# Setup Spark Session

In [ ]:
%%capture

!apt install libkrb5-dev
!pip install findspark
!pip install sparkmagic
!pip install pyspark
!pip install pyspark --user
!pip install seaborn --user
!pip install imageio --user
!pip install folium --user

!apt update
!apt install gcc python-dev libkrb5-dev

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import os

spark = SparkSession.builder.getOrCreate()

In [ ]:
%load_ext sparkmagic.magics

The sparkmagic.magics extension is already loaded. To reload it, use:
  %reload_ext sparkmagic.magics


In [ ]:
#graph section
import networkx as nx
# SQLite RDBMS
import sqlite3
# Parallel processing
# import swifter
import pandas as pd
# NoSQL DB
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

import os
os.environ['SPARK_HOME'] = '/content/spark-2.4.5-bin-hadoop2.7'
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
import pyspark
from pyspark.sql import SQLContext

try:
    if(spark == None):
        spark = SparkSession.builder.appName('Initial').getOrCreate()
        sqlContext=SQLContext(spark)
except NameError:
    spark = SparkSession.builder.appName('Initial').getOrCreate()
    sqlContext=SQLContext(spark)

We will load the data into the Spark context here.

In [ ]:
stock_data_sdf = spark.read.load(
    'stock_data/*.csv', 
    format = 'csv', 
    header = 'true', 
    inferSchema = 'true', 
    sep = ','
)

root
 |-- symbol: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- adjusted_close: double (nullable = true)
 |-- volume: long (nullable = true)



First we'll set up the spark dataframe for stock prices using the work in `step1a`.

In [ ]:
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import year, month, date_format


stock_data_sdf = stock_data_sdf.withColumn("timestamp_as_dt", to_timestamp(stock_data_sdf.timestamp, 'yyyy-MM-dd'))
stock_data_sdf = stock_data_sdf.withColumn("year", year('timestamp_as_dt'))
stock_data_sdf = stock_data_sdf.filter("year >= 2002 AND year <= 2019")

count_by_symbol_year_sdf = stock_data_sdf.groupBy(["symbol", "year"]).count()
count_years_by_symbol_sdf = count_by_symbol_year_sdf.groupBy(["symbol"]).count()

### Just grab stocks that have data in each of the 18 years from 2002 to 2019
### AND remove the three outliers
stocks_with_all_analysis_yrs_sdf = count_years_by_symbol_sdf.filter("count == 18") #18 years of data from 2002 and 2019
stocks_to_remove = ['DCTH', 'BRK-A', 'AIKI']
stocks_with_all_analysis_yrs_sdf = stocks_with_all_analysis_yrs_sdf.filter(stocks_with_all_analysis_yrs_sdf.symbol.isin(stocks_to_remove) == False)

stock_data_sdf.createOrReplaceTempView("stock_data")
stocks_with_all_analysis_yrs_sdf.createOrReplaceTempView("stocks_with_all_analysis_yrs")

stock_data_2002_2019_sdf = spark.sql(
    """
    SELECT *
    FROM stock_data
    WHERE symbol IN (SELECT symbol FROM stocks_with_all_analysis_yrs)
    """
)

In [ ]:
technical_data_sdf = spark.read.load(
    'technical_data/*.csv', 
    format = 'csv', 
    header = 'true', 
    inferSchema = 'true', 
    sep = ','
)

root
 |-- symbol: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- EMA: string (nullable = true)
 |-- MACD: string (nullable = true)
 |-- MACD_Hist: string (nullable = true)
 |-- MACD_Signal: string (nullable = true)
 |-- SlowD: string (nullable = true)
 |-- SlowK: string (nullable = true)
 |-- RSI: string (nullable = true)
 |-- Real_Lower_Band: string (nullable = true)
 |-- Real_Middle_Band: string (nullable = true)
 |-- Real_Upper_Band: double (nullable = true)



Then we will reduce the technical data set to just those stocks that match up with the first dataset above.

In [ ]:
technical_data_sdf = technical_data_sdf.withColumn("timestamp_as_dt", to_timestamp(technical_data_sdf.timestamp, 'yyyy-MM-dd'))
technical_data_sdf = technical_data_sdf.withColumn("year", year('timestamp_as_dt'))
technical_data_sdf = technical_data_sdf.filter("year >= 2002 AND year <= 2019")

technical_data_sdf.createOrReplaceTempView("technical_data")

technical_data_2002_2019_sdf = spark.sql(
    """
    SELECT *
    FROM technical_data
    WHERE symbol IN (SELECT symbol FROM stocks_with_all_analysis_yrs)
    """
)

Next step is to clean up technical data (especially fixing the schema so its numeric instead of string) and then setting it up for modeling.